# Approximate Nearest Neighbours with FAISS
In this notebook, we're going to learn how to do approximate nearest neighbours in FAISS using a cell probe method.

In [493]:
!pip install faiss-cpu pandas numpy

In [3]:
import faiss
import copy
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly_functions import generate_distinct_colors, zoom_in, create_plot, plot_points

In [4]:
dimensions = 2
number_of_vectors = 10_000
vectors = np.random.random((number_of_vectors, dimensions)).astype(np.float32)

In [5]:
fig = create_plot()
plot_points(fig, points=vectors, color='#CCCCCC', label="Data")
fig

In [6]:
search_vector = np.array([[0.5, 0.5]])

In [7]:
plot_points(fig, points=search_vector, color='black', label="Search Vector", symbol="x", size=10)
fig

## Creating a cell probe index
When creating the index, we need to specify how many partitions (or cells) we want to divide the vector space into.

In [8]:
cells = 10

quantizer = faiss.IndexFlatL2(dimensions)
index = faiss.IndexIVFFlat(quantizer, dimensions, cells)

In [9]:
index.train(vectors)

In [10]:
centroids = index.quantizer.reconstruct_n(0, index.nlist)
centroids

array([[0.8133438 , 0.13079676],
       [0.4825912 , 0.16446075],
       [0.1638229 , 0.89330345],
       [0.84360653, 0.4438653 ],
       [0.8327933 , 0.81593823],
       [0.13522407, 0.13323526],
       [0.536843  , 0.49398813],
       [0.20938276, 0.41296953],
       [0.18463777, 0.6608739 ],
       [0.50352746, 0.84820986]], dtype=float32)

## Visualising cells and centroids
Let's update our chart to show the centroids and to which cell each vector will be assigned.

In [11]:
_, cell_ids = index.quantizer.search(vectors, k=1)
cell_ids = cell_ids.flatten()
cell_ids[:10]

array([0, 1, 2, 1, 9, 6, 5, 0, 2, 1])

In [12]:
color_map = generate_distinct_colors(index.nlist)

fig_cells = create_plot()

unique_ids = np.unique(cell_ids)
for uid in unique_ids:
  mask = (cell_ids == uid)
  masked_vectors = vectors[mask]
  plot_points(fig_cells, masked_vectors, color_map[uid], "Cell {}".format(uid), size=6)

plot_points(fig_cells, centroids, symbol="diamond-tall", color="black", size=15, showlegend=False)
plot_points(fig_cells, search_vector, symbol="x", color="black", size=15, label="Search Vector")

fig_cells

In [ ]:
!pip install kaleido

In [13]:
import plotly.io as pio

current_width = fig_cells.layout.width
current_height = fig_cells.layout.height
print(current_width, current_height)


pio.orca.config.executable = '/usr/local/bin/orca'  # Replace with the actual path to your Orca executable


desired_width = 1600  # or whatever width you prefer in pixels
scaling_factor = desired_width / current_width if current_width else None

# If you know the current width, scale the height proportionally.
desired_height = current_height * scaling_factor if current_height and scaling_factor else None
fig_cells.update_layout(showlegend=False)
fig_cells.update_layout(template="plotly_white", margin=dict(t=0, b=0, l=0, r=0))
pio.write_image(fig_cells, 'figure_high_res.png', width=1280, height=720, scale=3.0)


None None


## Searching for our vector
Let's add the vectors to the index and look for our search vector.

In [14]:
index.add(vectors)

When using a cell probe index, we can specify how many cells we want to use in the search. More cells will mean a slower, but potentially more accurate search.

In [15]:
index.nprobe

1

In [16]:
%%time 
distances, indices = index.search(search_vector, k=10)

df_ann = pd.DataFrame({
  "id": indices[0],
  "vector": [vectors[id] for id in indices[0]],
  "distance": distances[0],
})
df_ann

CPU times: user 1.54 ms, sys: 2.7 ms, total: 4.24 ms
Wall time: 4.92 ms


,id,vector,distance
0,9815,"[0.50137174, 0.49799818]",0.000006
1,4313,"[0.4992051, 0.5044262]",0.000020
2,5096,"[0.5027816, 0.5039713]",0.000024
3,5008,"[0.49660861, 0.5045602]",0.000032
4,3350,"[0.50791544, 0.4995088]",0.000063
5,2831,"[0.50951856, 0.5008842]",0.000091
6,9874,"[0.50643706, 0.5115854]",0.000176
7,5156,"[0.51242346, 0.50469357]",0.000176
8,2851,"[0.50693977, 0.5114393]",0.000179
9,7233,"[0.48379314, 0.5034849]",0.000275


In [17]:
_, search_vectors_cell_ids = index.quantizer.search(search_vector, k=1)
unique_searched_ids = search_vectors_cell_ids[0]
unique_searched_ids

array([6])

In [18]:
fig_search = create_plot()

for uid in unique_searched_ids:
  mask = (cell_ids == uid)
  masked_vectors = vectors[mask]
  plot_points(fig_search, masked_vectors, color_map[uid], label="Cell {}".format(uid))
  plot_points(fig_search, centroids[uid].reshape(1, -1), symbol="diamond-tall", color="black", size=10, label="Centroid for Cell {}".format(uid), showlegend=False)

plot_points(fig_search, points=search_vector, color='black', label="Search Vector", symbol="x", size=10)

ann_vectors = np.array(df_ann["vector"].tolist())
plot_points(fig_search, points=ann_vectors, color='black', label="Approx Nearest Neighbors")

fig_search

# Brute Force Nearest Neighbours
How well did this approach work compared to a brute force one?

In [19]:
brute_force_index = faiss.IndexFlatL2(dimensions)
brute_force_index.add(vectors)

In [20]:
%%time
distances, indices = brute_force_index.search(search_vector, k=10)

pd.DataFrame({
  "id": indices[0],
  "vector": [vectors[id] for id in indices[0]],
  "distance": distances[0],
  "cell": [cell_ids[id] for id in indices[0]]
})

CPU times: user 1.79 ms, sys: 1.86 ms, total: 3.65 ms
Wall time: 3.98 ms


,id,vector,distance,cell
0,9815,"[0.50137174, 0.49799818]",0.000006,6
1,4313,"[0.4992051, 0.5044262]",0.000020,6
2,5096,"[0.5027816, 0.5039713]",0.000024,6
3,5008,"[0.49660861, 0.5045602]",0.000032,6
4,3350,"[0.50791544, 0.4995088]",0.000063,6
5,2831,"[0.50951856, 0.5008842]",0.000091,6
6,9874,"[0.50643706, 0.5115854]",0.000176,6
7,5156,"[0.51242346, 0.50469357]",0.000176,6
8,2851,"[0.50693977, 0.5114393]",0.000179,6
9,7233,"[0.48379314, 0.5034849]",0.000275,6


In [21]:
index.nprobe = 2

In [22]:
index.quantizer.search(search_vector, k=2)

(array([[0.00139355, 0.09203268]], dtype=float32), array([[6, 7]]))

In [23]:
%%time
distances, indices = index.search(search_vector, k=10)

pd.DataFrame({
  "id": indices[0],
  "vector": [vectors[id] for id in indices[0]],
  "distance": distances[0],
  "cell": [cell_ids[id] for id in indices[0]]
})

CPU times: user 905 μs, sys: 423 μs, total: 1.33 ms
Wall time: 885 μs


,id,vector,distance,cell
0,9815,"[0.50137174, 0.49799818]",0.000006,6
1,4313,"[0.4992051, 0.5044262]",0.000020,6
2,5096,"[0.5027816, 0.5039713]",0.000024,6
3,5008,"[0.49660861, 0.5045602]",0.000032,6
4,3350,"[0.50791544, 0.4995088]",0.000063,6
5,2831,"[0.50951856, 0.5008842]",0.000091,6
6,9874,"[0.50643706, 0.5115854]",0.000176,6
7,5156,"[0.51242346, 0.50469357]",0.000176,6
8,2851,"[0.50693977, 0.5114393]",0.000179,6
9,7233,"[0.48379314, 0.5034849]",0.000275,6
